In [1]:
#netstat -anp | grep 80 오류뜨면 실행중인 chorme 프로세스 삭제

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os
import shutil
import re
import time

options = Options()
options.add_argument("--single-process")
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')
#다운로드 설정
options.add_experimental_option("prefs", {
        "download.default_directory": r"/workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data",
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing_for_trusted_sources_enabled": False,
        "safebrowsing.enabled": False
})

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(2)
download_file_path = r"/workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data"

base_url = "http://www.gumi.go.kr/portal/bbs/list.do?ptIdx=164&mId=0305090000"


#게시판 다음 버튼 클릭하는 함수
def next_btn_click():
    driver.find_element_by_xpath('//*[@id="listForm"]/div[2]/strong/following-sibling::a').click()

#파일 이름을 과이름으로 변경
def file_rename(rename):       
    pattern = r'\((\S*)\)'
    rename = re.search(pattern, rename).group(1)
    filename = max([download_file_path + "/" + f for f in os.listdir(download_file_path)],key=os.path.getctime)
    if not(re.search(r'.xlsx$', filename)):
        print("엑셀형식이 아님", filename)
        os.remove(filename)
        return
    for i in range(1,100):
        if (os.path.isfile(os.path.join(download_file_path, f'{rename}{i}.xlsx'))):
            print("이미 존재하는 파일 목록입니다")
            continue
        shutil.move(filename,os.path.join(download_file_path, f'{rename}{i}.xlsx'))
        break;
    
    
#게시판에 있는 글 클릭후, 파일 다운로드하는 함수
def file_downloader_wrapper():
    list_num = 1
    def page_file_download():
        nonlocal list_num
        if(list_num == 11):
            next_btn_click()
            list_num = 1
        driver.find_element_by_xpath('//*[@id="listForm"]/table/tbody/tr[{0}]/td[2]/a'.format(list_num)).click()
        rename = driver.find_element_by_xpath('//*[@id="cnt"]/div/div[1]/h4').text
        try:
            driver.find_element_by_xpath('//*[@id="updateFileList"]/ul/li/a[1]').click()
            #속도가 너무 빠르면 crodownload(크롬 임시파일 확장자)로 저장되 오류가 다운완료까지 잠시 대기해줌
            time.sleep(0.5)
            file_rename(rename)
        except Exception as e:
            print("파일이 첨부되어있지 않습니다", e)
        driver.back()
        list_num += 1
    return page_file_download
    

def excel_parser(n):
    try:
        driver.get(base_url)
        file_downloader = file_downloader_wrapper()
        for i in range(1,n):
            if(i%100 == 0):
                print(f"{(i/n)*100}퍼센트 완료")
            file_downloader()
    except Exception as e:
        print(e)
    finally:
        driver.quit()
        

In [2]:
excel_parser(4000)

엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/2020.9월+업무추진비.xls
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/업무추진비+사용내역(총무과9월).hwp
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/2020년+9월+업무추진비+사용내역(농산물도매시장관리사무소).hwp
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/2020.9월+업무추진비+집행내역(대중교통과).hwp
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/업무추진비+집행내역(2020년+9월-세정과).xls
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/업무추진비+집행내역(2020년9월)_안전재난과.xls
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/□+하수과+업무추진비+사용내역(9월).hwp
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/업무추진비+집행내역(2020_9월).xls
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/9월+업무추진비+사용내역(선주원남동).xls
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/2020.9월+업무추진비(민원봉사과).hwp
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_da

이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/업무추진비+집행내역(2020년+6월)_안전재난과.xls
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/2020년+3월_5월+업무추진비+집행내역(도로과).xls
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/업무추진비집행내역(2020년5)_건설수변과.xls
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
엑셀형식이 아님 /w

엑셀형식이 아님 /workspace/Gumi/Gumi_famous_restaurant/receipt_excel_data/2020년+업무추진비+집행내역(3월).xls
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
이미 존재하는 파일 목록입니다
엑셀형식이 아님 /workspace/Gumi/Gumi_famous_rest